# [SOURCE] ART COMPARISON

In [1]:
import pandas as pd
import numpy as np
import pickle
from pandas.errors import EmptyDataError

import sys
sys.path.append("./scripts/modules")

from benchmarking_definitions import *
from regions import *

In [2]:
tagsRv = pd.read_csv("/home/sm624/projects/mixed_calls/benchmarking/comprehensive/FINAL_tag_list.csv", names=["tag"])["tag"].values
tagsL1234 = pd.read_csv("/home/sm624/projects/mixed_calls/benchmarking/comprehensive/FINAL_tag_list_L1234.csv", names=["tag"]).tag.values

len(tagsRv), len(tagsL1234)

(2500, 1000)

In [3]:
ART_Rv_mutant_summary_source = "/n/scratch/users/s/sm624/benchmarking/variant_summaries/ART_H37Rv"
ART_L1234_mutant_summary_source = "/n/scratch/users/s/sm624/benchmarking/variant_summaries/ART_L1234"

In [4]:
Rv_mpileup_source = "/n/data1/hms/dbmi/farhat/shandu/projects/mixed_calls/benchmarking/comprehensive/ART_pipeline_outputs/mpileup/output"
L1234_mpileup_source = "/n/data1/hms/dbmi/farhat/shandu/projects/mixed_calls/benchmarking/comprehensive/ARTL123_pipeline_outputs/mpileup/output"

# FPR per depth

In [4]:
depth_FPR_df = pd.DataFrame(columns=["tag", "depth", "genome", "tool", "num_FP", "FPR"])
df_i = 0

count = 0

genome = "H37Rv"

for tag in tagsRv:

    if count % 100 == 0:
        print(count, end=" ")
    count += 1

    depth = int(tag.split("_")[-2])

    variant_summary_df = pd.read_csv(f"{ART_Rv_mutant_summary_source}/{tag}.variant_summary.csv", low_memory=False)

    for tool in tools:

        tool_FP_df = variant_summary_df[(variant_summary_df.GT == 0) & (variant_summary_df[f"{tool}_found"] == 1)]

        tool_total_FP = tool_FP_df.shape[0]
        tool_FPR = tool_total_FP/genome_length

        depth_FPR_df.loc[df_i] = [tag, depth, genome, tool, tool_total_FP, tool_FPR]
        df_i += 1

genome = "L1-4"

for tag in tagsL1234:

    if count % 100 == 0:
        print(count, end=" ")
    count += 1

    depth = int(tag.split("_")[-2])
    
    variant_summary_df = pd.read_csv(f"{ART_L1234_mutant_summary_source}/{tag}.variant_summary.csv", low_memory=False)
    variant_summary_df = variant_summary_df[variant_summary_df.L_GT == 0]

    for tool in tools:

        tool_FP_df = variant_summary_df[(variant_summary_df.GT == 0) & (variant_summary_df[f"{tool}_found"] == 1)]
        tool_FP_df = tool_FP_df[tool_FP_df[f"{tool}_AF"] <= 0.50]

        tool_total_FP = tool_FP_df.shape[0]
        tool_FPR = tool_total_FP/genome_length

        depth_FPR_df.loc[df_i] = [tag, depth, genome, tool, tool_total_FP, tool_FPR]
        df_i += 1


0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 

In [5]:
print(depth_FPR_df.shape)
depth_FPR_df.head(2)

(21000, 6)


,tag,depth,genome,tool,num_FP,FPR
0,sim1_h37rv_mutant_1_50_0.01,50,H37Rv,FB,88,0.00002
1,sim1_h37rv_mutant_1_50_0.01,50,H37Rv,LF,0,0.00000


In [7]:
depth_FPR_df.to_csv("./data/ART_comparison/FPR_by_depth.csv", index=False)

# H37Rv weighted F1

In [8]:
scaling_factor = {"DR": 106, "HT": 1, "LM": 4}
scaling_factor_total = sum(scaling_factor.values())

In [9]:
Rv_weighted_F1_df = pd.DataFrame(columns=["tag", "simulation", "depth", "freq", "tool", "weighted_F1"])
df_i = 0

count = 0
for tag in tagsRv:
    
    if count % 10 == 0:
        print(count, end=" ")
    count += 1
    
    simulation = int(tag.split("_")[0].strip("sim"))
    depth = int(tag.split("_")[-2])
    freq = float(tag.split("_")[-1])
    
    mutant_summary_df = pd.read_csv("{0}/{1}.variant_summary.csv".format(ART_Rv_mutant_summary_source, tag), low_memory=False)
        
    for tool in tools:
        
        tool_TP = mutant_summary_df[(mutant_summary_df.GT == 1) & (mutant_summary_df["{}_found".format(tool)] == 1)]
        tool_FP = mutant_summary_df[(mutant_summary_df.GT == 0) & (mutant_summary_df["{}_found".format(tool)] == 1)]
        tool_FN = mutant_summary_df[(mutant_summary_df.GT == 1) & (mutant_summary_df["{}_found".format(tool)] == 0)]

        precision_values, recall_values = {}, {}
        
        for region in regions:

            region_TP = tool_TP[tool_TP.REGION_sim == region]
            region_FN = tool_FN[tool_FN.REGION_sim == region]
            region_FP = tool_FP[tool_FP.REGION_sim == region]

            num_region_TP = region_TP.shape[0]
            num_region_FN = region_FN.shape[0]
            num_region_FP = region_FP.shape[0]

            region_precision = calc_precision(num_region_TP, num_region_FP)
            region_recall = calc_recall(num_region_TP, num_region_FN)

            precision_values[region] = region_precision
            recall_values[region] = region_recall

        weighted_precision = sum([precision_values[r]*scaling_factor[r] for r in regions])/scaling_factor_total
        weighted_recall = sum([recall_values[r]*scaling_factor[r] for r in regions])/scaling_factor_total
        
        weighted_F1 = calc_F1(weighted_precision, weighted_recall)
            
        Rv_weighted_F1_df.loc[df_i] = [tag, simulation, depth, freq, tool, weighted_F1]
        df_i += 1
        

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 2020 2030 2040 2050 2060 2070 2080 2090 2100 2110 2120 2130 2140 2150 2160 2170 2180 2190 2200 2210 2

In [10]:
Rv_weighted_F1_df.head()

,tag,simulation,depth,freq,tool,weighted_F1
0,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,FB,0.255464
1,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,LF,0.000000
2,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,MT,0.095495
3,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,PL,0.185494
4,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,VD,0.239421


In [11]:
Rv_weighted_F1_df.to_csv("./data/ART_comparison/H37Rv_weighted_F1.csv", index=False)

# Average precision & recall by region

## H37Rv

In [12]:
Rv_precision_recall_by_region_df = pd.DataFrame(columns=["tag", "simulation", "depth", "freq", "genome", "tool", "region", "precision", "recall"])
df_i = 0

count = 0
for tag in tagsRv:
    
    if count % 10 == 0:
        print(count, end=" ")
    count += 1
    
    simulation = int(tag.split("_")[0].strip("sim"))
    depth = int(tag.split("_")[-2])
    freq = float(tag.split("_")[-1])
    
    mutant_summary_df = pd.read_csv("{0}/{1}.variant_summary.csv".format(ART_Rv_mutant_summary_source, tag), low_memory=False)
        
    for tool in tools:
        
        tool_TP = mutant_summary_df[(mutant_summary_df.GT == 1) & (mutant_summary_df["{}_found".format(tool)] == 1)]
        tool_FP = mutant_summary_df[(mutant_summary_df.GT == 0) & (mutant_summary_df["{}_found".format(tool)] == 1)]
        tool_FN = mutant_summary_df[(mutant_summary_df.GT == 1) & (mutant_summary_df["{}_found".format(tool)] == 0)]

        for region in regions:

            region_TP = tool_TP[tool_TP.REGION_sim == region]
            region_FN = tool_FN[tool_FN.REGION_sim == region]
            region_FP = tool_FP[tool_FP.REGION_sim == region]

            num_region_TP = region_TP.shape[0]
            num_region_FN = region_FN.shape[0]
            num_region_FP = region_FP.shape[0]

            region_precision = calc_precision(num_region_TP, num_region_FP)
            region_recall = calc_recall(num_region_TP, num_region_FN)

            Rv_precision_recall_by_region_df.loc[df_i] = [tag, simulation, depth, freq, "H37Rv", tool, region, region_precision, region_recall]
            df_i += 1
                                        

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 2020 2030 2040 2050 2060 2070 2080 2090 2100 2110 2120 2130 2140 2150 2160 2170 2180 2190 2200 2210 2

In [13]:
Rv_precision_recall_by_region_df.head()

,tag,simulation,depth,freq,genome,tool,region,precision,recall
0,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,H37Rv,FB,DR,0.75,0.157895
1,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,H37Rv,FB,HT,1.00,0.100000
2,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,H37Rv,FB,LM,1.00,0.050000
3,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,H37Rv,LF,DR,0.00,0.000000
4,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,H37Rv,LF,HT,0.00,0.000000


In [14]:
Rv_precision_recall_by_region_df.to_csv("./data/ART_comparison/H37Rv_precision_recall_by_region.csv", index=False)

## L1234 (DR only)

In [15]:
region = "DR"

L1234_DR_precision_recall_df = pd.DataFrame(columns=["tag", "simulation", "depth", "freq", "genome", "lineage", "tool", "region", "precision", "recall"])
df_i = 0

count = 0
for tag in tagsL1234:
    
    if count % 10 == 0:
        print(count, end=" ")
    count += 1
    
    simulation = int(tag.split("_")[0].strip("sim"))
    lineage = int(tag.split("_")[1].strip("L"))
    depth = int(tag.split("_")[-2])
    freq = float(tag.split("_")[-1])
    
    mutant_summary_df = pd.read_csv("{0}/{1}.variant_summary.csv".format(ART_L1234_mutant_summary_source, tag), low_memory=False)
    mutant_summary_nonbaseline_df = mutant_summary_df[mutant_summary_df.L_GT == 0]
    DR_mutant_summary_df = mutant_summary_nonbaseline_df[mutant_summary_nonbaseline_df.REGION_sim == region]
        
    for tool in tools:
        
        DR_tool_TP = DR_mutant_summary_df[(DR_mutant_summary_df.GT == 1) & (DR_mutant_summary_df["{}_found".format(tool)] == 1)]
        DR_tool_FN = DR_mutant_summary_df[(DR_mutant_summary_df.GT == 1) & (DR_mutant_summary_df["{}_found".format(tool)] == 0)]
        DR_tool_FP = DR_mutant_summary_df[(DR_mutant_summary_df.GT == 0) & (DR_mutant_summary_df["{}_found".format(tool)] == 1)]
        DR_tool_FP = DR_tool_FP[DR_tool_FP["{}_AF".format(tool)] <= 0.50]

        num_DR_TP = DR_tool_TP.shape[0]
        num_DR_FN = DR_tool_FN.shape[0]
        num_DR_FP = DR_tool_FP.shape[0]

        DR_precision = calc_precision(num_DR_TP, num_DR_FP)
        DR_recall = calc_recall(num_DR_TP, num_DR_FN)

        L1234_DR_precision_recall_df.loc[df_i] = [tag, simulation, depth, freq, "L1234", lineage, tool, region, DR_precision, DR_recall]
        df_i += 1
                                        

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 

In [16]:
L1234_DR_precision_recall_df.head()

,tag,simulation,depth,freq,genome,lineage,tool,region,precision,recall
0,sim1_L1_mutant_50_0.01,1,50,0.01,L1234,1,FB,DR,0.000000,0.000000
1,sim1_L1_mutant_50_0.01,1,50,0.01,L1234,1,LF,DR,0.000000,0.000000
2,sim1_L1_mutant_50_0.01,1,50,0.01,L1234,1,MT,DR,1.000000,0.210526
3,sim1_L1_mutant_50_0.01,1,50,0.01,L1234,1,PL,DR,1.000000,0.210526
4,sim1_L1_mutant_50_0.01,1,50,0.01,L1234,1,VD,DR,0.571429,0.210526


In [17]:
L1234_DR_precision_recall_df.to_csv("./data/ART_comparison/L1234_precision_recall_DR.csv", index=False)

# H37Rv + L1-4: F1 per region for all AFs

In [18]:
max_freq = 0.50

## Depths 50-200x

In [19]:
low_depths = [50, 100, 200]

In [20]:
for freq_i in range(len(freqs)):

    min_freq = freqs[freq_i]

    print(f"min_freq: {min_freq} >>", end=" ")

    min_freq_df = pd.DataFrame(columns=["tag", "depth", "tool", "region", "min_freq", "num_TP", "num_FN", "num_FP"])
    df_i = 0

    for sim_i in range(1, 6):

        for depth in low_depths:

            for freq_j in range(len(freqs) - freq_i):

                freq = freqs[freq_i+freq_j]
    
                for mutant_num in range(1, 11):

                    tag = f"sim{sim_i}_h37rv_mutant_{mutant_num}_{depth}_{freq}"
    
                    mutant_summary_df = pd.read_csv(f"{ART_Rv_mutant_summary_source}/{tag}.variant_summary.csv", low_memory=False)
                
                    for tool in tools:
                        
                        tool_TP = mutant_summary_df[(mutant_summary_df.GT == 1) & (mutant_summary_df[f"{tool}_found"] == 1)]
                        tool_FN = mutant_summary_df[(mutant_summary_df.GT == 1) & (mutant_summary_df[f"{tool}_found"] == 0)]

                        tool_FP_all = mutant_summary_df[(mutant_summary_df.GT == 0) & (mutant_summary_df[f"{tool}_found"] == 1)]
                        tool_FP = tool_FP_all[(tool_FP_all[f"{tool}_AF"] >= min_freq) & (tool_FP_all[f"{tool}_AF"] <= max_freq)]

                        for region in regions:
                
                            region_TP = tool_TP[tool_TP.REGION_sim == region].shape[0]
                            region_FN = tool_FN[tool_FN.REGION_sim == region].shape[0]
                            region_FP = tool_FP[tool_FP.REGION_sim == region].shape[0]

                            min_freq_df.loc[df_i] = [tag, depth, tool, region, min_freq, region_TP, region_FN, region_FP]
                            df_i += 1

                for lineage in range(1, 5):

                    tag = f"sim{sim_i}_L{lineage}_mutant_{depth}_{freq}"
    
                    mutant_summary_df = pd.read_csv(f"{ART_L1234_mutant_summary_source}/{tag}.variant_summary.csv", low_memory=False)
                    mutant_summary_df = mutant_summary_df[mutant_summary_df.L_GT == 0]
                
                    for tool in tools:
                        
                        tool_TP = mutant_summary_df[(mutant_summary_df.GT == 1) & (mutant_summary_df[f"{tool}_found"] == 1)]
                        tool_FN = mutant_summary_df[(mutant_summary_df.GT == 1) & (mutant_summary_df[f"{tool}_found"] == 0)]

                        tool_FP_all = mutant_summary_df[(mutant_summary_df.GT == 0) & (mutant_summary_df[f"{tool}_found"] == 1)]
                        tool_FP = tool_FP_all[(tool_FP_all[f"{tool}_AF"] >= min_freq) & (tool_FP_all[f"{tool}_AF"] <= max_freq)]
                
                        region_TP = tool_TP[tool_TP.REGION_sim == "DR"].shape[0]
                        region_FN = tool_FN[tool_FN.REGION_sim == "DR"].shape[0]
                        region_FP = tool_FP[tool_FP.REGION_sim == "DR"].shape[0]

                        min_freq_df.loc[df_i] = [tag, depth, tool, "DR", min_freq, region_TP, region_FN, region_FP]
                        df_i += 1
                    
    
    
    min_freq_df.to_csv(f"./data/ART_comparison/allAF_metrics_lowDepth_by_AF_minFreq{min_freq}.csv", index=False)
    print("written")


min_freq: 0.01 >> written
min_freq: 0.02 >> written
min_freq: 0.03 >> written
min_freq: 0.04 >> written
min_freq: 0.05 >> written
min_freq: 0.1 >> written
min_freq: 0.2 >> written
min_freq: 0.3 >> written
min_freq: 0.4 >> written
min_freq: 0.5 >> written


In [21]:
for region in regions:

    region_df = pd.DataFrame(columns=["min_freq", "precision", "recall", "F1", "tool"])
    region_df.loc[0] = [0, 0, 0, 0, "dummy"]

    for min_freq in freqs:

        min_freq_df = pd.read_csv(f"./data/ART_comparison/allAF_metrics_lowDepth_by_AF_minFreq{min_freq}.csv")

        region_min_freq_df = min_freq_df[min_freq_df.region == region]

        TP_lst = region_min_freq_df.num_TP
        FN_lst = region_min_freq_df.num_FN
        FP_lst = region_min_freq_df.num_FP

        precision_vals = [calc_precision(TP, FP) for TP,FP in zip(TP_lst, FP_lst)]
        recall_vals = [calc_precision(TP, FN) for TP,FN in zip(TP_lst, FN_lst)]

        F1_vals = [calc_F1(P, R) for P,R in zip(precision_vals, recall_vals)]

        add_df = pd.DataFrame(columns=region_df.columns)
        add_df["precision"] = precision_vals
        add_df["recall"] = recall_vals
        add_df["F1"] = F1_vals
        add_df["min_freq"] = region_min_freq_df.min_freq.values
        add_df["tool"] = region_min_freq_df.tool.values

        region_df = pd.concat([region_df, add_df])

    region_df.drop(0, inplace=True)

    region_df.to_csv(f"./data/ART_comparison/allAF_metrics_lowDepth_by_AF_{region}.csv", index=False)
        

# AF differences for each tool

In [37]:
AF_diffs = {t:[] for t in tools}

count = 0
for tag in tagsRv:
    
    count += 1
    if count % 10 == 0:
        print(count, end=" ")
    
    pileup_AF_df = pd.read_csv("{0}/{1}.AF.csv".format(Rv_mpileup_source, tag))
    
    tag_summary_df = pd.read_csv("{0}/{1}.variant_summary.csv".format(ART_Rv_mutant_summary_source, tag), low_memory=False).set_index(["POS", "ALT"])
    tag_summary_df = tag_summary_df[tag_summary_df.GT == 1]
        
    for i in pileup_AF_df.index:
        
        POS, ALT = pileup_AF_df.loc[i, "POS"], pileup_AF_df.loc[i, "ALT"]
        pileup_AF = pileup_AF_df.loc[i, "AF"]
        
        for tool in tools:
            
            tool_AF = tag_summary_df.loc[(POS, ALT), "{}_AF".format(tool)]
            
            if not np.isnan(tool_AF):
                
                tool_AF_diff = (tool_AF - pileup_AF)*100
                
                AF_diffs[tool].append(tool_AF_diff)

for tag in tagsL1234:
    
    count += 1
    if count % 10 == 0:
        print(count, end=" ")
    
    pileup_AF_df = pd.read_csv("{0}/{1}.AF.csv".format(L1234_mpileup_source, tag))
    
    tag_summary_df = pd.read_csv("{0}/{1}.variant_summary.csv".format(ART_L1234_mutant_summary_source, tag), low_memory=False).set_index(["POS", "ALT"])
    tag_summary_df = tag_summary_df[tag_summary_df.GT == 1]
    
    for i in pileup_AF_df.index:
        
        POS, ALT = pileup_AF_df.loc[i, "POS"], pileup_AF_df.loc[i, "ALT"]
        pileup_AF = pileup_AF_df.loc[i, "AF"]
        
        for tool in tools:
            
            tool_AF = tag_summary_df.loc[(POS, ALT), "{}_AF".format(tool)]
            
            if not np.isnan(tool_AF):
                
                tool_AF_diff = (tool_AF - pileup_AF)*100
                
                AF_diffs[tool].append(tool_AF_diff)
                    

10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 2020 2030 2040 2050 2060 2070 2080 2090 2100 2110 2120 2130 2140 2150 2160 2170 2180 2190 2200 2210 222

In [38]:
with open("./data/ART_comparison/AF_differences_by_tool_dict.pkl", "wb") as out_f:
    pickle.dump(AF_diffs, out_f)

# FP AF distribution per LM region

## H37Rv

In [24]:
count = 0

region_AF_distr_dict = {t:{"LM":[], "non-LM":[]} for t in tools}

for tag in tagsRv:

    if count % 100 == 0:
        print(count, end=" ")
    count += 1

    variant_summary_df = pd.read_csv(f"{ART_Rv_mutant_summary_source}/{tag}.variant_summary.csv", low_memory=False)

    for tool in tools:

        tool_FP_df = variant_summary_df[(variant_summary_df.GT == 0) & (variant_summary_df[f"{tool}_found"] == 1)]

        LM_tool_FP_df = tool_FP_df[tool_FP_df.REGION == "LM"]
        region_AF_distr_dict[tool]["LM"] += list(LM_tool_FP_df[f"{tool}_AF"].values*100)

        nonLM_tool_FP_df = tool_FP_df[tool_FP_df.REGION != "LM"]
        region_AF_distr_dict[tool]["non-LM"] += list(nonLM_tool_FP_df[f"{tool}_AF"].values*100)



0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 

In [25]:
with open(f"./data/ART_comparison/H37Rv_LM_region_AF_distr.pkl", "wb") as out_f:
    pickle.dump(region_AF_distr_dict, out_f)

## L1234

In [26]:
count = 0

region_AF_distr_dict = {t:{"LM":[], "non-LM":[]} for t in tools}

for tag in tagsL1234:

    if count % 100 == 0:
        print(count, end=" ")
    count += 1

    variant_summary_df = pd.read_csv(f"{ART_L1234_mutant_summary_source}/{tag}.variant_summary.csv", low_memory=False)
    variant_summary_df = variant_summary_df[variant_summary_df.L_GT == 0]

    for tool in tools:

        tool_FP_df = variant_summary_df[(variant_summary_df.GT == 0) & (variant_summary_df[f"{tool}_found"] == 1)]
        tool_FP_df = tool_FP_df[tool_FP_df[f"{tool}_AF"] <= 0.50]

        LM_tool_FP_df = tool_FP_df[tool_FP_df.REGION == "LM"]
        region_AF_distr_dict[tool]["LM"] += list(LM_tool_FP_df[f"{tool}_AF"].values*100)

        nonLM_tool_FP_df = tool_FP_df[tool_FP_df.REGION != "LM"]
        region_AF_distr_dict[tool]["non-LM"] += list(nonLM_tool_FP_df[f"{tool}_AF"].values*100)



0 100 200 300 400 500 600 700 800 900 

In [27]:
with open(f"./data/ART_comparison/L1234_LM_region_AF_distr.pkl", "wb") as out_f:
    pickle.dump(region_AF_distr_dict, out_f)

# FP AF distribution per DR region

## H37Rv

In [28]:
count = 0

DR_region_AF_distr_dict = {t:{"DR":[], "non-DR":[]} for t in tools}

for tag in tagsRv:

    if count % 100 == 0:
        print(count, end=" ")
    count += 1

    variant_summary_df = pd.read_csv(f"{ART_Rv_mutant_summary_source}/{tag}.variant_summary.csv", low_memory=False)

    for tool in tools:

        tool_FP_df = variant_summary_df[(variant_summary_df.GT == 0) & (variant_summary_df[f"{tool}_found"] == 1)]

        DR_tool_FP_df = tool_FP_df[tool_FP_df.REGION == "DR"]
        DR_region_AF_distr_dict[tool]["DR"] += list(DR_tool_FP_df[f"{tool}_AF"].values*100)

        nonDR_tool_FP_df = tool_FP_df[tool_FP_df.REGION != "DR"]
        DR_region_AF_distr_dict[tool]["non-DR"] += list(nonDR_tool_FP_df[f"{tool}_AF"].values*100)



0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 

In [29]:
with open(f"./data/ART_comparison/H37Rv_DR_region_AF_distr.pkl", "wb") as out_f:
    pickle.dump(DR_region_AF_distr_dict, out_f)

## L1234

In [30]:
count = 0

DR_region_AF_distr_dict = {t:{"DR":[], "non-DR":[]} for t in tools}

for tag in tagsL1234:

    if count % 100 == 0:
        print(count, end=" ")
    count += 1

    variant_summary_df = pd.read_csv(f"{ART_L1234_mutant_summary_source}/{tag}.variant_summary.csv", low_memory=False)
    variant_summary_df = variant_summary_df[variant_summary_df.L_GT == 0]

    for tool in tools:

        tool_FP_df = variant_summary_df[(variant_summary_df.GT == 0) & (variant_summary_df[f"{tool}_found"] == 1)]
        tool_FP_df = tool_FP_df[tool_FP_df[f"{tool}_AF"] <= 0.50]

        DR_tool_FP_df = tool_FP_df[tool_FP_df.REGION == "DR"]
        DR_region_AF_distr_dict[tool]["DR"] += list(DR_tool_FP_df[f"{tool}_AF"].values*100)

        nonDR_tool_FP_df = tool_FP_df[tool_FP_df.REGION != "DR"]
        DR_region_AF_distr_dict[tool]["non-DR"] += list(nonDR_tool_FP_df[f"{tool}_AF"].values*100)



0 100 200 300 400 500 600 700 800 900 

In [31]:
with open(f"./data/ART_comparison/L1234_DR_region_AF_distr.pkl", "wb") as out_f:
    pickle.dump(DR_region_AF_distr_dict, out_f)

# L1234: FP AF distribution in DR vs LM vs OTHER regions

In [5]:
count = 0

all_region_AF_distr_dict = {t:{"DR_nonLM":[], "other_nonLM":[], "LM": []} for t in tools}

for tag in tagsL1234:

    if count % 100 == 0:
        print(count, end=" ")
    count += 1

    variant_summary_df = pd.read_csv(f"{ART_L1234_mutant_summary_source}/{tag}.variant_summary.csv", low_memory=False)
    FP_df = variant_summary_df[(variant_summary_df.L_GT == 0) & (variant_summary_df.GT == 0)]

    DR_nonLM_FP_df = FP_df[FP_df.POS.isin(DR_pos) & ~FP_df.POS.isin(LM_pos)]
    other_nonLM_FP_df = FP_df[~FP_df.POS.isin(DR_pos) & ~FP_df.POS.isin(LM_pos)]
    LM_FP_df = FP_df[FP_df.POS.isin(LM_pos)]

    for tool in tools:

        tool_DR_nonLM_FP_df = DR_nonLM_FP_df[(DR_nonLM_FP_df[f"{tool}_found"] == 1) & (DR_nonLM_FP_df[f"{tool}_AF"] <= 0.50)]
        all_region_AF_distr_dict[tool]["DR_nonLM"] += list(tool_DR_nonLM_FP_df[f"{tool}_AF"].values*100)

        tool_other_nonLM_FP_df = other_nonLM_FP_df[(other_nonLM_FP_df[f"{tool}_found"] == 1) & (other_nonLM_FP_df[f"{tool}_AF"] <= 0.50)]
        all_region_AF_distr_dict[tool]["other_nonLM"] += list(tool_other_nonLM_FP_df[f"{tool}_AF"].values*100)

        tool_LM_FP_df = LM_FP_df[(LM_FP_df[f"{tool}_found"] == 1) & (LM_FP_df[f"{tool}_AF"] <= 0.50)]
        all_region_AF_distr_dict[tool]["LM"] += list(tool_LM_FP_df[f"{tool}_AF"].values*100)



0 100 200 300 400 500 600 700 800 900 

In [6]:
with open(f"./data/ART_comparison/L1234_FP_AF_distr_DR_LM_other.pkl", "wb") as out_f:
    pickle.dump(all_region_AF_distr_dict, out_f)

# L1234: FPR in DR vs LM vs OTHER regions

In [5]:
DR_pos_arr = np.array(list(DR_pos.keys()))
LM_pos_arr = np.array(LM_pos)

DR_or_LM_pos_arr = np.union1d(DR_pos_arr, LM_pos_arr)
genome_pos_arr = np.arange(1, genome_length+1)

DR_nonLM_pos = DR_pos_arr[~np.isin(DR_pos_arr, LM_pos_arr)]
other_nonLM_pos = genome_pos_arr[~np.isin(genome_pos_arr, DR_or_LM_pos_arr)]

In [6]:
all_region_lengths = {"DR_nonLM": len(DR_nonLM_pos),
                      "other_nonLM": len(other_nonLM_pos),
                      "LM": len(LM_pos)}

In [7]:
L1234_region_FPR_df = pd.DataFrame(columns=["tag", "region", "tool", "num_FP", "FPR"])
df_i = 0

count = 0
for tag in tagsL1234:

    if count % 100 == 0:
        print(count, end=" ")
    count += 1
    
    variant_summary_df = pd.read_csv(f"{ART_L1234_mutant_summary_source}/{tag}.variant_summary.csv", low_memory=False)
    FP_df = variant_summary_df[(variant_summary_df.L_GT == 0) & (variant_summary_df.GT == 0)]

    DR_nonLM_FP_df = FP_df[FP_df.POS.isin(DR_pos) & ~FP_df.POS.isin(LM_pos)]
    other_nonLM_FP_df = FP_df[~FP_df.POS.isin(DR_pos) & ~FP_df.POS.isin(LM_pos)]
    LM_FP_df = FP_df[FP_df.POS.isin(LM_pos)]

    region_FP_dfs = {"DR_nonLM": DR_nonLM_FP_df,
                  "other_nonLM": other_nonLM_FP_df,
                  "LM": LM_FP_df}

    for region in region_FP_dfs:

        region_FP_df = region_FP_dfs[region]

        region_length = all_region_lengths[region]

        for tool in tools:

            tool_region_FP_df = region_FP_df[(region_FP_df[f"{tool}_found"] == 1) & (region_FP_df[f"{tool}_AF"] <= 0.50)]

            tool_region_num_FP = tool_region_FP_df.shape[0]

            tool_region_FPR = tool_region_num_FP/region_length

            L1234_region_FPR_df.loc[df_i] = [tag, region, tool, tool_region_num_FP, tool_region_FPR]
            df_i += 1



0 100 200 300 400 500 600 700 800 900 

In [8]:
print(L1234_region_FPR_df.shape)
L1234_region_FPR_df.head(2)

(18000, 5)


,tag,region,tool,num_FP,FPR
0,sim1_L1_mutant_50_0.01,DR_nonLM,FB,0,0.0
1,sim1_L1_mutant_50_0.01,DR_nonLM,LF,0,0.0


In [9]:
L1234_region_FPR_df.to_csv("./data/ART_comparison/L1234_FPR_DR_LM_other.csv", index=False)

# H37Rv: FPR in DR vs LM vs OTHER regions

In [5]:
DR_pos_arr = np.array(list(DR_pos.keys()))
LM_pos_arr = np.array(LM_pos)

DR_or_LM_pos_arr = np.union1d(DR_pos_arr, LM_pos_arr)
genome_pos_arr = np.arange(1, genome_length+1)

DR_nonLM_pos = DR_pos_arr[~np.isin(DR_pos_arr, LM_pos_arr)]
other_nonLM_pos = genome_pos_arr[~np.isin(genome_pos_arr, DR_or_LM_pos_arr)]

In [6]:
all_region_lengths = {"DR_nonLM": len(DR_nonLM_pos),
                      "other_nonLM": len(other_nonLM_pos),
                      "LM": len(LM_pos)}

In [7]:
Rv_region_FPR_df = pd.DataFrame(columns=["tag", "region", "tool", "num_FP", "FPR"])
df_i = 0

count = 0
for tag in tagsRv:

    if count % 100 == 0:
        print(count, end=" ")
    count += 1
    
    variant_summary_df = pd.read_csv(f"{ART_Rv_mutant_summary_source}/{tag}.variant_summary.csv", low_memory=False)
    FP_df = variant_summary_df[variant_summary_df.GT == 0]

    DR_nonLM_FP_df = FP_df[FP_df.POS.isin(DR_pos) & ~FP_df.POS.isin(LM_pos)]
    other_nonLM_FP_df = FP_df[~FP_df.POS.isin(DR_pos) & ~FP_df.POS.isin(LM_pos)]
    LM_FP_df = FP_df[FP_df.POS.isin(LM_pos)]

    region_FP_dfs = {"DR_nonLM": DR_nonLM_FP_df,
                  "other_nonLM": other_nonLM_FP_df,
                  "LM": LM_FP_df}

    for region in region_FP_dfs:

        region_FP_df = region_FP_dfs[region]

        region_length = all_region_lengths[region]

        for tool in tools:

            tool_region_FP_df = region_FP_df[region_FP_df[f"{tool}_found"] == 1]

            tool_region_num_FP = tool_region_FP_df.shape[0]

            tool_region_FPR = tool_region_num_FP/region_length

            Rv_region_FPR_df.loc[df_i] = [tag, region, tool, tool_region_num_FP, tool_region_FPR]
            df_i += 1



0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 

In [8]:
print(Rv_region_FPR_df.shape)
Rv_region_FPR_df.head(2)

(45000, 5)


,tag,region,tool,num_FP,FPR
0,sim1_h37rv_mutant_1_50_0.01,DR_nonLM,FB,1,0.000027
1,sim1_h37rv_mutant_1_50_0.01,DR_nonLM,LF,0,0.000000


In [9]:
Rv_region_FPR_df.to_csv("./data/ART_comparison/H37Rv_FPR_DR_LM_other.csv", index=False)

# L1234: FPR per region

In [32]:
all_region_lengths = {"DR": DR_region_length,
                      "HT": HT_region_length,
                      "LM": LM_region_length,
                      "other": genome_length-DR_region_length-HT_region_length-LM_region_length}

In [33]:
L1234_region_FPR_df = pd.DataFrame(columns=["tag", "region", "tool", "num_FP", "FPR"])
df_i = 0

count = 0
for tag in tagsL1234:

    if count % 100 == 0:
        print(count, end=" ")
    count += 1
    
    variant_summary_df = pd.read_csv(f"{ART_L1234_mutant_summary_source}/{tag}.variant_summary.csv", low_memory=False)
    variant_summary_df = variant_summary_df[variant_summary_df.L_GT == 0]

    for tool in tools:

        tool_FP_df = variant_summary_df[(variant_summary_df.GT == 0) & (variant_summary_df[f"{tool}_found"] == 1)]
        tool_FP_df = tool_FP_df[tool_FP_df[f"{tool}_AF"] <= 0.50]

        for region in regions_other:

            region_tool_num_FP = tool_FP_df[tool_FP_df.REGION == region].shape[0]

            region_length = all_region_lengths[region]

            region_tool_FPR = region_tool_num_FP/region_length

            L1234_region_FPR_df.loc[df_i] = [tag, region, tool, region_tool_num_FP, region_tool_FPR]
            df_i += 1


0 100 200 300 400 500 600 700 800 900 

In [34]:
print(L1234_region_FPR_df.shape)
L1234_region_FPR_df.head(2)

(24000, 5)


,tag,region,tool,num_FP,FPR
0,sim1_L1_mutant_50_0.01,DR,FB,0,0.0
1,sim1_L1_mutant_50_0.01,HT,FB,0,0.0


In [35]:
L1234_region_FPR_df.to_csv("./data/ART_comparison/L1234_region_FPR.csv", index=False)

# Genome-wide FPR in H37Rv vs L1234

In [39]:
genome_FPR_df = pd.DataFrame(columns=["tag", "genome", "tool", "num_FP", "FPR"])
df_i = 0

count = 0

genome = "H37Rv"

for tag in tagsRv:

    if count % 100 == 0:
        print(count, end=" ")
    count += 1

    variant_summary_df = pd.read_csv(f"{ART_Rv_mutant_summary_source}/{tag}.variant_summary.csv", low_memory=False)

    for tool in tools:

        tool_FP_df = variant_summary_df[(variant_summary_df.GT == 0) & (variant_summary_df[f"{tool}_found"] == 1)]

        tool_total_FP = tool_FP_df.shape[0]
        tool_FPR = tool_total_FP/genome_length

        genome_FPR_df.loc[df_i] = [tag, genome, tool, tool_total_FP, tool_FPR]
        df_i += 1

genome = "L1-4"

for tag in tagsL1234:

    if count % 100 == 0:
        print(count, end=" ")
    count += 1
    
    variant_summary_df = pd.read_csv(f"{ART_L1234_mutant_summary_source}/{tag}.variant_summary.csv", low_memory=False)
    variant_summary_df = variant_summary_df[variant_summary_df.L_GT == 0]

    for tool in tools:

        tool_FP_df = variant_summary_df[(variant_summary_df.GT == 0) & (variant_summary_df[f"{tool}_found"] == 1)]
        tool_FP_df = tool_FP_df[tool_FP_df[f"{tool}_AF"] <= 0.50]

        tool_total_FP = tool_FP_df.shape[0]
        tool_FPR = tool_total_FP/genome_length

        genome_FPR_df.loc[df_i] = [tag, genome, tool, tool_total_FP, tool_FPR]
        df_i += 1


0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 

In [40]:
print(genome_FPR_df.shape)
genome_FPR_df.head()

(21000, 5)


,tag,genome,tool,num_FP,FPR
0,sim1_h37rv_mutant_1_50_0.01,H37Rv,FB,88,0.000020
1,sim1_h37rv_mutant_1_50_0.01,H37Rv,LF,0,0.000000
2,sim1_h37rv_mutant_1_50_0.01,H37Rv,MT,0,0.000000
3,sim1_h37rv_mutant_1_50_0.01,H37Rv,PL,31,0.000007
4,sim1_h37rv_mutant_1_50_0.01,H37Rv,VD,330,0.000075


In [41]:
genome_FPR_df.to_csv("./data/ART_comparison/genome_wide_FPR.csv", index=False)

# Percentage of total FP that are LM

In [15]:
percent_LM_FP_df = pd.DataFrame(columns=["tag", "genome", "tool", "num_FP", "num_LM_FP", "percent_LM_FP"])
df_i = 0

count = 0

genome = "H37Rv"
for tag in tagsRv:

    if count % 100 == 0:
        print(count, end=" ")
    count += 1

    variant_summary_df = pd.read_csv(f"{ART_Rv_mutant_summary_source}/{tag}.variant_summary.csv", low_memory=False)

    for tool in tools:

        tool_FP_df = variant_summary_df[(variant_summary_df.GT == 0) & (variant_summary_df[f"{tool}_found"] == 1)]
        tool_total_FP = tool_FP_df.shape[0]

        LM_tool_total_FP = tool_FP_df[tool_FP_df.REGION == "LM"].shape[0]

        if tool_total_FP > 0:
            percent_LM_FP = (LM_tool_total_FP/tool_total_FP)*100            

            percent_LM_FP_df.loc[df_i] = [tag, genome, tool, tool_total_FP, LM_tool_total_FP, percent_LM_FP]
            df_i += 1

genome = "L1-4"
for tag in tagsL1234:

    if count % 100 == 0:
        print(count, end=" ")
    count += 1

    variant_summary_df = pd.read_csv(f"{ART_L1234_mutant_summary_source}/{tag}.variant_summary.csv", low_memory=False)
    variant_summary_df = variant_summary_df[variant_summary_df.L_GT == 0]

    for tool in tools:

        tool_FP_df = variant_summary_df[(variant_summary_df.GT == 0) & (variant_summary_df[f"{tool}_found"] == 1)]
        tool_FP_df = tool_FP_df[tool_FP_df[f"{tool}_AF"] <= 0.50]
        tool_total_FP = tool_FP_df.shape[0]

        LM_tool_total_FP = tool_FP_df[tool_FP_df.REGION == "LM"].shape[0]

        if tool_total_FP > 0:
            percent_LM_FP = (LM_tool_total_FP/tool_total_FP)*100            

            percent_LM_FP_df.loc[df_i] = [tag, genome, tool, tool_total_FP, LM_tool_total_FP, percent_LM_FP]
            df_i += 1


0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 

In [16]:
print(percent_LM_FP_df.shape)
percent_LM_FP_df.head(2)

(13476, 6)


,tag,genome,tool,num_FP,num_LM_FP,percent_LM_FP
0,sim1_h37rv_mutant_1_50_0.01,H37Rv,FB,88,11,12.500000
1,sim1_h37rv_mutant_1_50_0.01,H37Rv,PL,31,4,12.903226


In [17]:
percent_LM_FP_df.to_csv("./data/ART_comparison/percent_LM_FP.csv", index=False)

# Percentage of total FP at AF = 1%

In [18]:
percent_AF1_FP_df = pd.DataFrame(columns=["tag", "genome", "tool", "num_FP", "num_AF1_FP", "percent_AF1_FP"])
df_i = 0

count = 0

genome = "H37Rv"
for tag in tagsRv:

    if count % 100 == 0:
        print(count, end=" ")
    count += 1

    variant_summary_df = pd.read_csv(f"{ART_Rv_mutant_summary_source}/{tag}.variant_summary.csv", low_memory=False)

    for tool in tools:

        tool_FP_df = variant_summary_df[(variant_summary_df.GT == 0) & (variant_summary_df[f"{tool}_found"] == 1)]
        tool_total_FP = tool_FP_df.shape[0]

        AF1_tool_total_FP = tool_FP_df[tool_FP_df[f"{tool}_AF"] == 0.01].shape[0]

        if tool_total_FP > 0:
            percent_AF1_FP = (AF1_tool_total_FP/tool_total_FP)*100            

            percent_AF1_FP_df.loc[df_i] = [tag, genome, tool, tool_total_FP, AF1_tool_total_FP, percent_AF1_FP]
            df_i += 1

genome = "L1-4"
for tag in tagsL1234:

    if count % 100 == 0:
        print(count, end=" ")
    count += 1

    variant_summary_df = pd.read_csv(f"{ART_L1234_mutant_summary_source}/{tag}.variant_summary.csv", low_memory=False)
    variant_summary_df = variant_summary_df[variant_summary_df.L_GT == 0]

    for tool in tools:

        tool_FP_df = variant_summary_df[(variant_summary_df.GT == 0) & (variant_summary_df[f"{tool}_found"] == 1)]
        tool_FP_df = tool_FP_df[tool_FP_df[f"{tool}_AF"] <= 0.50]
        tool_total_FP = tool_FP_df.shape[0]

        AF1_tool_total_FP = tool_FP_df[tool_FP_df[f"{tool}_AF"] == 0.01].shape[0]

        if tool_total_FP > 0:
            percent_AF1_FP = (AF1_tool_total_FP/tool_total_FP)*100            

            percent_AF1_FP_df.loc[df_i] = [tag, genome, tool, tool_total_FP, AF1_tool_total_FP, percent_AF1_FP]
            df_i += 1


0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 

In [19]:
print(percent_AF1_FP_df.shape)
percent_AF1_FP_df.head(2)

(13476, 6)


,tag,genome,tool,num_FP,num_AF1_FP,percent_AF1_FP
0,sim1_h37rv_mutant_1_50_0.01,H37Rv,FB,88,0,0.0
1,sim1_h37rv_mutant_1_50_0.01,H37Rv,PL,31,0,0.0


In [20]:
percent_AF1_FP_df.to_csv("./data/ART_comparison/percent_AF1_FP.csv", index=False)